In [1]:
# import librarys required for web scraping tools
from bs4 import BeautifulSoup
from datetime import datetime
import sys
import os
import psutil
from time import strftime
#from urllib import urlopen
import urllib3
import certifi
import numpy as np
import re
import csv
import pandas as pd

In [2]:
# current date and time
now = datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
time = now.strftime("%H:%M:%S")
date_time = "\\"+now.strftime("%Y%m%d%H%M")
#print("date and time:",date_time)

thisdir="C:\\Users\\eliza\\Dropbox\\work\\_VU\\HackyHourPrj\\Library"
#specify web address for Scimago SJR - *OPEN ACCESS ONLY*
SJRpathspecs="https://www.scimagojr.com/journalrank.php?openaccess=true"
xlsSJRpaths="https://www.scimagojr.com/journalrank.php?openaccess=true&out=xls"

#Also publisher websites:
SpringNat=["https://www.springeropen.com/journals",
          "https://www.biomedcentral.com/journals",
          "https://www.nature.com/openresearch/palgrave-journals",
           "https://www.nature.com/openresearch/publishing-with-npg/nature-journals"]
Wiley=["https://authorservices.wiley.com/open-research/open-access/browse-journals.html",
      "https://authorservices.wiley.com/author-resources/Journal-Authors/open-access/onlineopen.html"]
TayFranc=["https://www.tandfonline.com/openaccess/openjournals"]
PLOS=["https://www.plos.org/publications"]


#Also note Scimago ID pages:
#https://www.scimagojr.com/journalsearch.php?q=12137&tip=sid

In [3]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
page = http.request('GET', SJRpathspecs) # import the page we are interested in (e.g. dateusq)
soup = BeautifulSoup(page.data,"lxml")

# find entries associated with "All subject areas"
divTagALL=soup.findAll('ul', attrs = {'class' :'dropdown-options dropdown-element'})  
divTag=divTagALL[0]

countLAB=sum('</a>' in str(s) for s in divTag)
print(countLAB)
subLAB=[]
i=0
for tag in divTag.find_all("a"):
    i=+1
    subSTR="{0}".format(tag.text)
    subLAB.append(subSTR)

print(subLAB)

28
['All subject areas', 'Agricultural and Biological Sciences', 'Arts and Humanities', 'Biochemistry, Genetics and Molecular Biology', 'Business, Management and Accounting', 'Chemical Engineering', 'Chemistry', 'Computer Science', 'Decision Sciences', 'Dentistry', 'Earth and Planetary Sciences', 'Economics, Econometrics and Finance', 'Energy', 'Engineering', 'Environmental Science', 'Health Professions', 'Immunology and Microbiology', 'Materials Science', 'Mathematics', 'Medicine', 'Multidisciplinary', 'Neuroscience', 'Nursing', 'Pharmacology, Toxicology and Pharmaceutics', 'Physics and Astronomy', 'Psychology', 'Social Sciences', 'Veterinary']


In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options 
import time
from webdriver_manager.chrome import ChromeDriverManager
#install webdrivers using
#pip install webdriver-manager
options = webdriver.ChromeOptions() 
prefs = {'download.default_directory' : thisdir+date_time}
options.add_experimental_option('prefs', prefs)
#The same can be used to set Firefox, Edge
browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)
browser.get(xlsSJRpaths)
time.sleep(20)
browser.quit()


Checking for win32 chromedriver:76.0.3809.126 in cache
Driver found in C:\Users\eliza\.wdm\chromedriver\76.0.3809.126\win32/chromedriver.exe


In [5]:
newestdir=max(next(os.walk(thisdir))[1])
fname=os.listdir(thisdir+"//"+newestdir)[0]

In [6]:
#clean data can go straight into pandas
SCImago = pd.read_csv(thisdir+"\\"+newestdir+"\\"+fname,delimiter=";")
#print SCImago JRN if it is Q1
isQ1=SCImago['SJR Best Quartile']=="Q1"
SCImagoQ1data=SCImago[isQ1]
#SCImagoQ1data

In [7]:
catlist=[]
cattemplist=[]
for index, row in SCImagoQ1data.iterrows():
    templist=list(map(str, SCImagoQ1data['Categories'][index].split(";")))
    cattemplist= [jrnname for jrnname in templist if "Q1" in jrnname] 
    catlist.append(cattemplist)
#ignore warning    
pd.set_option('mode.chained_assignment', None)    
SCImagoQ1data['Q1CatList'] = list(catlist)  
SCImagoQ1data=SCImagoQ1data.drop(columns=['Categories'])

In [8]:
# resets the option for warnings we set in the previous code segment
pd.reset_option('mode.chained_assignment')
SCImagoQ1data

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2018),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Coverage,Q1CatList
0,1,19434,MMWR. Recommendations and reports : Morbidity ...,journal,"10575987, 15458601","48,894",Q1,134,3,12,559,1043,12,"86,00","186,33",United States,Centers for Disease Control and Prevention (CDC),1990-ongoing,"[Epidemiology (Q1), Health Information Manage..."
1,2,71056,MMWR. Surveillance summaries : Morbidity and m...,journal,"15458636, 15460738","18,375",Q1,90,14,56,851,1452,55,"37,11","60,79",United States,Centers for Disease Control and Prevention (CDC),2002-ongoing,"[Epidemiology (Q1), Health Information Manage..."
2,3,12137,Journal of Statistical Software,journal,15487660,"17,569",Q1,115,69,265,2878,6630,265,"10,30","41,71",United States,University of California at Los Angeles,1996-ongoing,"[Software (Q1), Statistics and Probability (Q..."
3,4,40944,Genome Biology,journal,"1474760X, 14747596","9,867",Q1,207,229,803,12124,8933,709,"12,16","52,94",United Kingdom,BioMed Central,2000-ongoing,"[Cell Biology (Q1), Ecology, Evolution, Behav..."
4,5,14204,Nucleic Acids Research,journal,"03051048, 13624962","8,636",Q1,452,1169,4058,57749,45067,3975,"11,14","49,40",United Kingdom,Oxford University Press,1974-ongoing,[Genetics (Q1)]
5,6,110092,MMWR. Morbidity and mortality weekly report,journal,"01492195, 1545861X","7,695",Q1,182,363,1171,2144,8568,679,"14,40","5,91",United States,US Department of Health and Human Services,1981-ongoing,"[Epidemiology (Q1), Health Information Manage..."
6,7,6200180159,World Psychiatry,journal,"20515545, 17238617","7,559",Q1,70,99,279,2985,2271,106,"24,03","30,15",United States,Wiley-Blackwell,2007-ongoing,"[Psychiatric Mental Health (Q1), Psychiatry a..."
7,8,24769,Materials Today,journal,"13697021, 18734103","7,392",Q1,138,94,512,10020,3628,154,"19,50","106,60",Netherlands,Elsevier BV,2002-ongoing,"[Condensed Matter Physics (Q1), Materials Sci..."
8,9,21100265444,The Lancet Global Health,journal,2214109X,"7,367",Q1,53,326,842,5302,3755,204,"16,37","16,26",United Kingdom,Elsevier BV,2013-ongoing,[Medicine (miscellaneous) (Q1)]
9,10,28677,Living Reviews in Relativity,journal,14338351,"7,304",Q1,75,7,13,3063,256,13,"24,20","437,57",Germany,Albert Einstein Institut,2000-ongoing,[Physics and Astronomy (miscellaneous) (Q1)]


Now investigate the publishers associated with these journals

In [9]:
allID=SCImagoQ1data.Sourceid.unique()
allweb4ID=[]

for IDnum in allID:
    newadd="https://www.scimagojr.com/journalsearch.php?q="+str(IDnum)+"&tip=sid"
    allweb4ID.append(newadd)
print(allweb4ID)

['https://www.scimagojr.com/journalsearch.php?q=19434&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=71056&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=12137&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=40944&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=14204&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=110092&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=6200180159&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=24769&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=21100265444&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=28677&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=6400153126&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=4700152228&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=21100201068&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=144840&tip=sid', 'https://www.scimagojr.com/journalsearch.php?q=21100242814&tip=sid', 'https://www.scimag

In [10]:
"""
This is now working and takes a long time to run (not an error). -> CHECK OUT THE PROGRESS BAR
"""
#tqdm is progress bar
from tqdm import tqdm
jrnPUB=[]
jrnPUBtemp=[]
tempVAL=[]
for curl in tqdm(range(len(allweb4ID))):
    tempcallurl=[]
    #sometimes you may run this code while the specification are being updated, or the websites have crashed.
    #the try and exception here is to alert you to any individual specifications which may be affected.
    #print(allweb4ID[curl])
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    page = http.request('GET', allweb4ID[curl])       
    soup = BeautifulSoup(page.data,"lxml") 
    #extract publisher title
    jrninfo=soup.find("div", {"class": "journaldescription colblock"})
    #print(jrninfo)
    td = jrninfo.find(lambda t: t.text.strip()=='Publisher').parent.select('td')
    for name, value in zip(td, td[1:]):
        tdtemp="".join(str(td))
        if re.search('journalsearch.php\?q=(.*)&', tdtemp).group(1) =="":
            jrnPUBtemp=[allID[curl],""]
        else: 
            cleanval=value.text.replace(",", "")
            jrnPUBtemp=[allID[curl],cleanval.strip()]

    jrnPUB.append(jrnPUBtemp)

100%|██████████| 1015/1015 [33:43<00:00,  2.01s/it]


In [11]:
jrnPUB

[[19434, 'Centers for Disease Control and Prevention (CDC)'],
 [71056, 'Centers for Disease Control and Prevention (CDC)'],
 [12137, 'University of California at Los Angeles'],
 [40944, 'BioMed Central'],
 [14204, 'Oxford University Press'],
 [110092, 'US Department of Health and Human Services'],
 [6200180159, 'Wiley-Blackwell'],
 [24769, 'Elsevier BV'],
 [21100265444, 'Elsevier BV'],
 [28677, 'Albert Einstein Institut'],
 [6400153126, 'Wiley-Blackwell'],
 [4700152228, 'Wiley-Blackwell'],
 [21100201068, 'Cell Press'],
 [144840, 'Public Library of Science'],
 [21100242814, 'eLife Sciences Publications'],
 [19700173023, 'Max-Planck-Institut fur Sonnensystemforschung'],
 [21100201772, 'American Physical Society'],
 [21100797856, 'Taylor and Francis Ltd.'],
 [21100457028, 'American Association for the Advancement of Science'],
 [19700182758, 'Nature Publishing Group'],
 [21100804406, 'Elsevier Limited'],
 [21100278313, 'Nature Publishing Group'],
 [21100284918, 'Copernicus Publications'],

In [21]:
jrnPUB[-100:]

[[86891, 'King Saud University'],
 [21100228569, 'Springer Science and Business Media Deutschland GmbH'],
 [21100411756, 'Elsevier BV'],
 [19700181203, 'Hosokawa Powder Technology Foundation'],
 [19700183006,
  'International Journal of Agricultural and Biological Engineering (IJABE)'],
 [21100202950, 'Universidad de Salamanca'],
 [5700165306, 'Nova Southeastern University'],
 [3200147837, 'Elsevier (Singapore) Pte Ltd'],
 [21100324700, 'Kuwait University'],
 [21100344997, 'SAGE Publications Ltd'],
 [145257,
  'Technology Education Program Virginia Polytechnic Institute and State University'],
 [5600153223, 'Instituto Universitario de Estudios sobre Migraciones'],
 [23836, 'De Gruyter Mouton'],
 [21100416147, 'Elsevier Limited'],
 [21100829268, 'Multidisciplinary Digital Publishing Institute (MDPI)'],
 [21100413834, 'Elsevier BV'],
 [14000156125, 'University of Hawaii Press'],
 [13427, 'De Gruyter Open Ltd.'],
 [21100372856, 'Elsevier BV'],
 [18659, 'Academia Brasileira de Ciencias'],


In [12]:
#save as csv
JRNfname=thisdir+"\\"+"jrnPUBlist"+now.strftime("%Y")+".csv"
   
csv_rows = ["{},{}".format(i, j) for i, j in np.asarray(jrnPUB)]
csv_text = "\n".join(csv_rows)

# write it to a file
with open(JRNfname, 'w') as f:
    f.write(csv_text)    

In [49]:
jrnPUBlist=pd.read_csv(JRNfname, sep=",", encoding ='cp1252' ,header=None)
#print(jrnPUBlist[:][1])
SCImagoQ1data['Publisher'] = jrnPUBlist[1].to_numpy()

In [50]:
SCImagoQ1data

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2018),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Coverage,Q1CatList
0,1,19434,MMWR. Recommendations and reports : Morbidity ...,journal,"10575987, 15458601","48,894",Q1,134,3,12,559,1043,12,"86,00","186,33",United States,Centers for Disease Control and Prevention (CDC),1990-ongoing,"[Epidemiology (Q1), Health Information Manage..."
1,2,71056,MMWR. Surveillance summaries : Morbidity and m...,journal,"15458636, 15460738","18,375",Q1,90,14,56,851,1452,55,"37,11","60,79",United States,Centers for Disease Control and Prevention (CDC),2002-ongoing,"[Epidemiology (Q1), Health Information Manage..."
2,3,12137,Journal of Statistical Software,journal,15487660,"17,569",Q1,115,69,265,2878,6630,265,"10,30","41,71",United States,University of California at Los Angeles,1996-ongoing,"[Software (Q1), Statistics and Probability (Q..."
3,4,40944,Genome Biology,journal,"1474760X, 14747596","9,867",Q1,207,229,803,12124,8933,709,"12,16","52,94",United Kingdom,BioMed Central,2000-ongoing,"[Cell Biology (Q1), Ecology, Evolution, Behav..."
4,5,14204,Nucleic Acids Research,journal,"03051048, 13624962","8,636",Q1,452,1169,4058,57749,45067,3975,"11,14","49,40",United Kingdom,Oxford University Press,1974-ongoing,[Genetics (Q1)]
5,6,110092,MMWR. Morbidity and mortality weekly report,journal,"01492195, 1545861X","7,695",Q1,182,363,1171,2144,8568,679,"14,40","5,91",United States,US Department of Health and Human Services,1981-ongoing,"[Epidemiology (Q1), Health Information Manage..."
6,7,6200180159,World Psychiatry,journal,"20515545, 17238617","7,559",Q1,70,99,279,2985,2271,106,"24,03","30,15",United States,Wiley-Blackwell,2007-ongoing,"[Psychiatric Mental Health (Q1), Psychiatry a..."
7,8,24769,Materials Today,journal,"13697021, 18734103","7,392",Q1,138,94,512,10020,3628,154,"19,50","106,60",Netherlands,Elsevier BV,2002-ongoing,"[Condensed Matter Physics (Q1), Materials Sci..."
8,9,21100265444,The Lancet Global Health,journal,2214109X,"7,367",Q1,53,326,842,5302,3755,204,"16,37","16,26",United Kingdom,Elsevier BV,2013-ongoing,[Medicine (miscellaneous) (Q1)]
9,10,28677,Living Reviews in Relativity,journal,14338351,"7,304",Q1,75,7,13,3063,256,13,"24,20","437,57",Germany,Albert Einstein Institut,2000-ongoing,[Physics and Astronomy (miscellaneous) (Q1)]
